# Trending YouTube Content Exploratory Data Analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
    <ul>
    <li><a href="#assess">Assess the data</a></li>
        <ul>
            <li><a href="#assess_sum">Assessment summary</a></li>
        </ul>
    <li><a href="#clean">Clean the data</a></li>
        <ul>
            <li><a href="clean_sum">Cleaning summary</a></li>
        </ul>
    </ul>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction
The purpose of this analysis is to analyze the YouTube trending content in the US. Below are a few questions I am looking to answer:
- What types of videos appear in trending content most often? How has that changed over time?
- Are there particular days/times videos were posted that affect top video performance?
- How long does it take for a video to become trending typically?
- Is there a threshold for the amount of engagement/other statistic to become trending?
- What creators have had the most success in consistently publishing trending video? Why is that?
- Are there common themes in best practices (titles, descriptions, tags) amongst trending content?

<a id='wrangling'></a>
## Data Wrangling

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

pio.templates.default = "plotly_white"
%matplotlib inline

In [2]:
# Read csv file of trending YouTube video data
videos = pd.read_csv(r'/Users/baka_brooks/Documents/my-projects/youtube/datasets/USvideos.csv')

In [3]:
# Read json file of US video categories
import json
from pandas.io.json import json_normalize

data = pd.read_json(r"/Users/baka_brooks/Documents/my-projects/youtube/datasets/US_category_id.json")

category_id = []
category = []
for item in data['items']:
    category_id.append(item['id'])
    category.append(item['snippet']['title'])
    
categories = pd.DataFrame(list(zip(category_id, category)), columns=['category_id', 'category'])
categories['category_id'] = categories['category_id'].astype('int64')
categories = categories.sort_values('category_id')

<a id='assess'></a>
### Assess the data

In [4]:
videos.head(5)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


- Tags separated by "|" character
- Description also contains special characters
- Trending date and published time columns are in different formats, and both are not the standard datetime format

In [5]:
videos.shape

(40949, 16)

- There are 40,949 videos included in this dataset and 16 columns

In [6]:
videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

- There are some null descriptions

In [7]:
videos.dtypes

video_id                  object
trending_date             object
title                     object
channel_title             object
category_id                int64
publish_time              object
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_count              int64
thumbnail_link            object
comments_disabled           bool
ratings_disabled            bool
video_error_or_removed      bool
description               object
dtype: object

- Dates and times are read as strings

In [8]:
videos.duplicated().sum()

48

- There are 48 duplicate rows in the dataset.

In [9]:
videos.video_id.nunique()

6351

- There are 40K+ rows in the dataset, but there are 6,351 unique videos featured.

In [10]:
videos.video_id.value_counts().head()

j4KvrAUjn6c    30
QBL8IRJ5yHU    29
NBSAQenU2Bk    29
t4pRQ0jn23Q    29
MAjY8mCTXWk    29
Name: video_id, dtype: int64

- The most common `video_id` is "j4KvrAUjn6c". Let's examine these rows to see the differences between these rows.

In [11]:
videos[videos['video_id'] == 'j4KvrAUjn6c']

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
34550,j4KvrAUjn6c,18.14.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",2131702,54280,4696,10705,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! TIX ON SALE NOW!! https://www.d...
34752,j4KvrAUjn6c,18.15.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",3906727,77378,12160,15874,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
34901,j4KvrAUjn6c,18.15.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",3906727,77378,12160,15874,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
34966,j4KvrAUjn6c,18.16.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",4862649,90451,14792,18035,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
35180,j4KvrAUjn6c,18.17.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",5665445,101461,16170,19395,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
35396,j4KvrAUjn6c,18.18.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",6290796,110373,17131,20365,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
35600,j4KvrAUjn6c,18.19.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",6773604,116508,17787,20941,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
35804,j4KvrAUjn6c,18.20.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",7204004,121673,18323,21459,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
36035,j4KvrAUjn6c,18.21.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",7512530,125317,18633,21812,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...
36245,j4KvrAUjn6c,18.22.05,WE MADE OUR MOM CRY...HER DREAM CAME TRUE!,Lucas and Marcus,24,2018-05-13T18:03:56.000Z,"Lucas and Marcus|""Marcus and Lucas""|""Dobre""|""D...",7740671,128377,18901,22052,https://i.ytimg.com/vi/j4KvrAUjn6c/default.jpg,False,False,False,BEST MOM EVER! WANT TO SEE US IN NYC & NJ?!BUY...


- The non-duplicate rows that are of the same video vary in trending date.

In [12]:
categories.head(5)

,category_id,category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


- Some categories appear to be missing in the dataset

In [13]:
categories.dtypes

category_id     int64
category       object
dtype: object

In [14]:
categories.category.value_counts()

Comedy                   2
Foreign                  1
Travel & Events          1
Trailers                 1
News & Politics          1
Education                1
People & Blogs           1
Shows                    1
Anime/Animation          1
Entertainment            1
Family                   1
Shorts                   1
Videoblogging            1
Documentary              1
Gaming                   1
Music                    1
Sci-Fi/Fantasy           1
Science & Technology     1
Classics                 1
Movies                   1
Sports                   1
Film & Animation         1
Autos & Vehicles         1
Nonprofits & Activism    1
Pets & Animals           1
Short Movies             1
Thriller                 1
Drama                    1
Horror                   1
Action/Adventure         1
Howto & Style            1
Name: category, dtype: int64

In [15]:
categories.query('category == "Comedy"')

,category_id,category
10,23,Comedy
21,34,Comedy


<a id='assess_sum'></a>
**Assessment summary**
1. The `trending_date` and `publish_time` columns are strings and will need to be converted to datetime objects.
2. The `trending_date` column is in the wrong format to be read as a datetime object.
3. The `publish_time` column includes both date and time. In order to provide more segmented time-based analysis, date and time will be split into separate columns.
4. There are 48 duplicate rows in the dataset. These rows will be removed.
5. Merge the two DataFrames on `category_id`.
6. There are gaps between the category id numbers. Monitor this when merging dataframes to determine if any additional data needs to be collected.
7. There are two "Comedy" categories. This will need to be assessed after merging the DataFrames to be sure that there is not mis-classification.
8. Rename the `comment_count` column to `columns`, for consistency and ease of analysis.
9. Of the 40K+ videos featured in the dataset, there are 6,351 unique videos. The other rows represent different dates in which the video was trending. A separate DataFrame will be created after merging the video categories in order to analyze individual videos based on the first date they were trending.

<a id='clean'></a>
### Clean the data

In [16]:
videos_clean = videos.copy()
categories_clean = categories.copy()

1. The `trending_date` and `publish_time` columns are strings and will need to be converted to datetime objects.   
For the purposes of this analysis, I will only need to change the data types of the date and time columns.

2. The `trending_date` column is in the wrong format to be read as a datetime object.   
The `trending_date` column is currently in an unreadable date format, so I will need to get the data into a suitable format first.

In [17]:
videos_clean["publish_time"] = pd.to_datetime(videos_clean["publish_time"])
videos_clean.dtypes

video_id                               object
trending_date                          object
title                                  object
channel_title                          object
category_id                             int64
publish_time              datetime64[ns, UTC]
tags                                   object
views                                   int64
likes                                   int64
dislikes                                int64
comment_count                           int64
thumbnail_link                         object
comments_disabled                        bool
ratings_disabled                         bool
video_error_or_removed                   bool
description                            object
dtype: object

In [18]:
videos_clean.trending_date.value_counts()

18.05.04    200
17.25.11    200
18.04.05    200
17.17.12    200
17.10.12    200
           ... 
18.31.01    197
18.01.02    197
18.04.02    196
18.03.02    196
18.02.02    196
Name: trending_date, Length: 205, dtype: int64

- By observation, all of the records are in the yy.dd.mm format, so I will parse the date this way across the entire column. 

In [19]:
(videos_clean.publish_time.min(), videos_clean.publish_time.max())

(Timestamp('2006-07-23 08:24:11+0000', tz='UTC'),
 Timestamp('2018-06-14 01:31:53+0000', tz='UTC'))

- The dataset ranges from the years 2006 to 2018, so I will concatenate "20" to the year of the cleaned trending date.

In [20]:
videos_clean["trending_date"] = videos_clean.trending_date.str.split(".")

In [21]:
date = []
for x in videos_clean.trending_date:
    year = "20" + x[0]
    day = x[1]
    month = x[2]
    date.append(year + "-" + month + "-" + day)

videos_clean["trending_date"] = date

In [22]:
videos_clean["trending_date"] = pd.to_datetime(videos_clean["trending_date"])

In [23]:
videos_clean.dtypes

video_id                               object
trending_date                  datetime64[ns]
title                                  object
channel_title                          object
category_id                             int64
publish_time              datetime64[ns, UTC]
tags                                   object
views                                   int64
likes                                   int64
dislikes                                int64
comment_count                           int64
thumbnail_link                         object
comments_disabled                        bool
ratings_disabled                         bool
video_error_or_removed                   bool
description                            object
dtype: object

3. The `publish_time` column includes both date and time. In order to provide more segmented time-based analysis, date and time will be split into separate columns.   
I will split the `publish_time` column to separate date and time to perform separate time series analyses.

In [24]:
videos_clean['publish_date'] = videos_clean['publish_time'].dt.date
videos_clean['publish_time_new'] = videos_clean['publish_time'].dt.time
videos_clean.head(2)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,publish_time_new
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,2017-11-13,17:13:01
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",2017-11-13,07:30:00


In [25]:
videos_clean.dtypes

video_id                               object
trending_date                  datetime64[ns]
title                                  object
channel_title                          object
category_id                             int64
publish_time              datetime64[ns, UTC]
tags                                   object
views                                   int64
likes                                   int64
dislikes                                int64
comment_count                           int64
thumbnail_link                         object
comments_disabled                        bool
ratings_disabled                         bool
video_error_or_removed                   bool
description                            object
publish_date                           object
publish_time_new                       object
dtype: object

- The `publish_date` and `publish_time_new` columns were created as strings and need to be converted back to datetime obejcts.

In [26]:
videos_clean['publish_date'] = pd.to_datetime(videos_clean['publish_date'])
videos_clean['publish_time_new'] = pd.to_timedelta(videos_clean['publish_time_new'])
videos_clean.dtypes

ValueError: Invalid type for timedelta scalar: <class 'datetime.time'>

In [ ]:
videos_clean.publish_time_new.value_counts()

- Drop the original `publish_time` column and rename the new time column

In [ ]:
videos_clean = videos_clean.drop(columns='publish_time', axis=1)
videos_clean = videos_clean.rename(columns={'publish_time_new': 'publish_time'})
videos_clean.head(2)

4. There are 48 duplicate rows in the dataset. These rows will be removed.   
This will be done using the `.drop_duplicates()` method.

In [ ]:
videos_clean = videos_clean.drop_duplicates()
videos_clean.shape

In [ ]:
videos_clean.duplicated('video_id').sum()

5. Merge the two DataFrames on `category_id`.
6. There are gaps between the category id numbers. Monitor this when merging dataframes to determine if any additional data needs to be collected.
7. There are two "Comedy" categories. This will need to be assessed after merging the DataFrames to be sure that there is not mis-classification. <br>
Merge the DataFrames first, then assess effect on categorization and clean further if necessary.

In [ ]:
# Merge DataFrames
df1 = videos_clean.merge(categories_clean, on='category_id', how='left')
df1.head()

In [ ]:
df1.groupby(['category_id', 'category'])['video_id'].count()

- There were no NaN values created in the `category` column as a result of the left join, so all videos were classified appropriately. Only one 'Comedy' category was needed.

8. Rename the `comment_count` column to `columns`, for consistency and ease of analysis.
I am also going to rename the "comment_count" column to "comments", just for my own sanity.

In [ ]:
df1 = df1.rename(columns={"comment_count": "comments"})
df1.head()

9. Of the 40K+ videos featured in the dataset, there are 6,351 unique videos. The other rows represent different dates in which the video was trending. A separate DataFrame will be created after merging the video categories in order to analyze individual videos based on the first date they were trending. <br>
I will need to find the `video_id` of all videos that trended on multiple days, find the day they were first trending, and create a DataFrame comprised of individual videos with the `trending_date` and associated stats of the day they first appeared in the trending section.

In [ ]:
df1.video_id.value_counts()

- The new DataFrame must filter out the duplicate videos, leaving only the row of the first trending date.

In [ ]:
df2 = df1.sort_values('trending_date').drop_duplicates('video_id', keep='last')
df2.shape

In [ ]:
df2.sort_values('video_id')

In [ ]:
df2.video_id.nunique()

- Now I will do a final check of data to be sure cleaning is complete.

In [ ]:
df1.dtypes

In [ ]:
df2.dtypes

- `publish_date` and `publish_time` need to be converted to datetime objects again in both dataframes

In [ ]:
df2['publish_date'] = pd.to_datetime(df2['publish_date'])
df2.dtypes

<a id='clean_sum'></a>
**Cleaning summary**
1. The date/time based columns were converted to datetime objects from strings using `pd.to_datetime()`.
2. The `trending_date` column was not in 'YYYY-MM-DD' format and needed to be split on '.' and concatenated in order to be converted to datetime.
3. The `publish_time` column includes both date and time and was split into two separate columns, `publish_date` and `publish_time` using pandas datetime methods.
4. 48 duplicate rows were dropped using `.drop_duplicates()`   
5. The `categories` DataFrame was left joined onto `videos` on `category_id`.
6. No NaN values were created in the `category` columns as a result of the left join, so no further cleaning needed to be done as a result of the join.
7. Only one `category_id` of the 'Comedy' category lived in the DataFrame, so there was no need to determine misclassification.
8. The `comment_count` column was renamed to `columns`, for consistent naming convention across other numeric columns.
9. The 6,351 unique videos were isolated into a separate DataFrame using their first trending date for categorical analysis.

<a id='eda'></a>
## Exploratory Data Analysis
Data types have been corrected, so now I can visually explore the data.

In [ ]:
df1.head(1)

In [ ]:
sns.pairplot(df1, vars=['likes', 'comments', 'dislikes']);

In [ ]:
category_order = df2.category.value_counts().index
fig, ax = plt.subplots()
plot = sns.countplot(data=df2, y='category', color='orange', order=category_order)
plt.xticks(rotation=90);

In [ ]:
# Calculate the number of days each video was trending into a separate column

# Create separate DataFrames containing the unique videos by minimum and maximum trending date
min_date = df1.loc[df1.groupby('video_id')['trending_date'].idxmin()]
max_date = df1.loc[df1.groupby('video_id')['trending_date'].idxmax()]

In [ ]:
# Converting the quantitative columns to logs to view distributions
log_views = np.log(df1["views"])
log_likes = np.log(df1["likes"])
log_dislikes = np.log(df1["dislikes"])
log_comments = np.log(df1["comments"])

plt.figure(figsize=(12,6))

plt.subplot(2,2,1)
ax1 = sns.distplot(log_views)

plt.subplot(2,2,2)
ax2 = sns.distplot(log_likes)

plt.subplot(2,2,3)
# ax3 = sns.distplot(log_dislikes)

plt.subplot(2,2,4)
# ax4 = sns.distplot(log_comments)

plt.subplots_adjust(wspace=0.2, hspace=0.4, top=0.9);

<a id='conclusions'></a>
## Conclusions
1. 
2. 
3. 
4. 

### Limitations
...